# Reto Pandas 1



Texto del reto propuesto en el grupo de telegram Python para Tranding:

Buenos días. Os propongo un reto / ejercicio de Pandas, para quien quiera practicar y mejorar.

con el dataframe data que obtenemos con este código

    # Si no está instalado el módulo yfinance puedes hacerlo con 
    # pip install yfinance

    import yfinance as yf
    data = yf.download('^GSPC')


Añadir una nueva columna donde el valor sea 1 los días en que el mes al que pertenece tuvo cotización el último día natural de ese mes, y -1 si no hubo cotización, por vacaciones o fin de semanas.

Así por ejemplo todos los días de cotización del mes de enero de 2020, llevarían un 1, mientras que los días de noviembre de 2019, llevarían un -1.

Hay varías formas de conseguir esto, veamos cual nos gusta más. Obviamente se puede hacer sin bucles usando pandas 😁😉

## Código base

In [5]:
# Si no está instalado el módulo yfinance puedes hacerlo con 
# pip install yfinance

import yfinance as yf
import pandas as pd
data_base = yf.download('^GSPC')

[*********************100%***********************]  1 of 1 completed


# Propuestas :

## Pau @pauet7 - 1ª 

In [2]:
import calendar
import datetime
data = data_base.copy()

In [3]:
%%timeit
data['last_open_day'] = data.index.map(lambda x: datetime.datetime(x.year, x.month, calendar.monthrange(x.year, x.month)[1]))
data['signal'] = data.apply(lambda x: 1 if x['last_open_day'] in data.index else -1, axis=1)

643 ms ± 19.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
data['signal']

Date
1927-12-30   -1
1928-01-03    1
1928-01-04    1
1928-01-05    1
1928-01-06    1
             ..
2020-02-03   -1
2020-02-04   -1
2020-02-05   -1
2020-02-06   -1
2020-02-07   -1
Name: signal, Length: 23135, dtype: int64

## Pau @pauet7 -  2ª

In [12]:
data = data_base.copy()

In [8]:
%%timeit
data['last_open_day'] = data.index.map(lambda x: pd.to_datetime(x) + pd.tseries.offsets.MonthEnd(0))
data['signal'] = data.apply(lambda x: 1 if x['last_open_day'] in data.index else -1, axis=1)

531 ms ± 19.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
data['signal']

Date
1927-12-30   -1
1928-01-03    1
1928-01-04    1
1928-01-05    1
1928-01-06    1
             ..
2020-02-03   -1
2020-02-04   -1
2020-02-05   -1
2020-02-06   -1
2020-02-07   -1
Name: signal, Length: 23135, dtype: int64

In [7]:
data = data_base.copy()

In [19]:
%%timeit 
data['last_open_day'] = data.index.map(lambda x: pd.to_datetime(x) + pd.tseries.offsets.MonthEnd(0))
data['signal'] = data.apply(lambda x: 1 if x['last_open_day'] in data.index else -1, axis=1)

543 ms ± 17.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Antonio A @init_S - 1ª

In [10]:
data = data_base.copy()

In [11]:
data.loc[:,'rounded_time'] = data.index.to_period('M')
data.loc[:, 'signal'] = data['rounded_time'].apply(lambda x: 1 if x in data[data.index.is_month_end].index.to_period('M') else -1)

In [15]:
data['signal']

Date
1927-12-30   -1
1928-01-03    1
1928-01-04    1
1928-01-05    1
1928-01-06    1
             ..
2020-02-03   -1
2020-02-04   -1
2020-02-05   -1
2020-02-06   -1
2020-02-07   -1
Name: signal, Length: 23135, dtype: int64

## Antonio A @init_S - 2ª

In [13]:
import numpy as np
data = data_base.copy()

In [14]:
%%timeit 
data.loc[:,'rounded_time'] = data.index.to_period('M')
data.loc[:, 'signal'] = np.where(data['rounded_time'].isin(data[data.index.is_month_end].index.to_period('M')), 1, -1)

61.9 ms ± 535 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [16]:
data['signal']

Date
1927-12-30   -1
1928-01-03    1
1928-01-04    1
1928-01-05    1
1928-01-06    1
             ..
2020-02-03   -1
2020-02-04   -1
2020-02-05   -1
2020-02-06   -1
2020-02-07   -1
Name: signal, Length: 23135, dtype: int64

## Dario @Bukosabino - 1ª

In [17]:
data = data_base.copy()

In [18]:
%%timeit
last_day_month_market = data.groupby([data.index.year, data.index.month]).tail(1).index
data['signal']=pd.DataFrame(index=last_day_month_market, 
                            data=last_day_month_market.is_month_end.astype(int)).reindex(data.index).fillna(method='backfill')[0]
data['signal']=np.where(data['signal']==0,-1,data['signal'])

9.29 ms ± 268 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [19]:
data['signal']

Date
1927-12-30   -1.0
1928-01-03    1.0
1928-01-04    1.0
1928-01-05    1.0
1928-01-06    1.0
             ... 
2020-02-03   -1.0
2020-02-04   -1.0
2020-02-05   -1.0
2020-02-06   -1.0
2020-02-07   -1.0
Name: signal, Length: 23135, dtype: float64

## Dario @Bukosabino - 2ª

In [20]:
data = data_base.copy()

In [21]:
%%timeit
last_day_month_market = data.groupby([data.index.year, data.index.month]).tail(1).index
data['signal']=pd.Series(index=last_day_month_market, data=last_day_month_market.is_month_end.astype(int)).reindex(data.index).fillna(method='backfill').values
data['signal']=np.where(data['signal']==0,-1,data['signal'])

10.2 ms ± 752 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [24]:
data['signal']

Date
1927-12-30   -1
1928-01-03    1
1928-01-04    1
1928-01-05    1
1928-01-06    1
             ..
2020-02-03   -1
2020-02-04   -1
2020-02-05   -1
2020-02-06   -1
2020-02-07   -1
Name: signal, Length: 23135, dtype: int64

## Paduel @paduel - 1ª

In [22]:
data = data_base.copy()

In [23]:
%%timeit
data['signal'] = data.index
data['signal'] = data.signal.groupby([data.index.year, data.index.month]).transform('last').dt.is_month_end.mul(2).sub(1)

12.8 ms ± 1.34 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [25]:
%%timeit
data['signal'] = data.index.to_frame().Date.groupby([data.index.year, data.index.month]).transform('last').dt.is_month_end.mul(2).sub(1)

13.5 ms ± 1.35 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [306]:
data['signal']

Date
1927-12-30   -1
1928-01-03    1
1928-01-04    1
1928-01-05    1
1928-01-06    1
             ..
2020-02-03   -1
2020-02-04   -1
2020-02-05   -1
2020-02-06   -1
2020-02-07   -1
Name: signal, Length: 23135, dtype: int64

## Paduel @paduel - 2ª

In [28]:
data = data_base.copy()

In [29]:
%%timeit
_d = data.index.to_frame().groupby([data.index.year, data.index.month]).last().squeeze()
data['signal'] = pd.Series(_d.dt.is_month_end.mul(2).sub(1).values, index=_d.values).reindex(data.index).bfill()

8.97 ms ± 162 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [30]:
data['signal']

Date
1927-12-30   -1.0
1928-01-03    1.0
1928-01-04    1.0
1928-01-05    1.0
1928-01-06    1.0
             ... 
2020-02-03   -1.0
2020-02-04   -1.0
2020-02-05   -1.0
2020-02-06   -1.0
2020-02-07   -1.0
Name: signal, Length: 23135, dtype: float64

# Bonus

Todos los métodos anteriores dan un -1 a los días del mes en curso cuando aún no tenemos datos para saber si el último día habrá o no cotización. Por lo que habría que añadir un código que corrigiera eso.
A continuación una propuesta one liner que evita ese problema.

## Paduel @paduel 3º

In [31]:
data = data_base.copy()

In [36]:
%%timeit
data['signal'] = data.Close.reindex(pd.date_range(data.index[0], data.index[-1], freq='M')).gt(0).replace(False,-1).reindex(pd.date_range(data.index[0], data.index[-1], freq='D')).bfill().reindex(data.index)

22.6 ms ± 1.74 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [34]:
data['signal']

Date
1927-12-30   -1.0
1928-01-03    1.0
1928-01-04    1.0
1928-01-05    1.0
1928-01-06    1.0
             ... 
2020-02-03    NaN
2020-02-04    NaN
2020-02-05    NaN
2020-02-06    NaN
2020-02-07    NaN
Name: signal, Length: 23135, dtype: float64

In [357]:
data['signal'].tail(10)

Date
2020-01-27    1.0
2020-01-28    1.0
2020-01-29    1.0
2020-01-30    1.0
2020-01-31    1.0
2020-02-03    NaN
2020-02-04    NaN
2020-02-05    NaN
2020-02-06    NaN
2020-02-07    NaN
Name: signal, dtype: float64